# Collection of Twitter data for the <font color='red'>#Day-zero</font> sentiment ananlysis
Data is scraped from twitter relating to the waer  crisis issue in Cape Town

- drought sort of began in 2015 in the Western cape
- Cape town Details:
    - -33.9249° S, 18.4241° E
- Countries we do business in:


### Loading Packages

In [53]:
import numpy as np
import pandas as pd
import tweepy as tpy
# import jsonpickle

ModuleNotFoundError: No module named 'jsonpickle'

### Authotrization

In [124]:
# import access keys
from credentials import *

# API setup
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tpy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    # Return API with authentication:
    api = tpy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)  # todo: include rate limit parameters
    return api

api = twitter_setup()

In [6]:
api.geo_id('df51dec6f4ee2b2c')

Place(_api=<tweepy.api.API object at 0x000001D647D236D8>, id='df51dec6f4ee2b2c', url='https://api.twitter.com/1.1/geo/id/df51dec6f4ee2b2c.json', place_type='neighborhood', name='Presidio', full_name='Presidio, San Francisco', country_code='US', country='United States', contained_within=[Place(_api=<tweepy.api.API object at 0x000001D647D236D8>, id='5a110d312052166f', url='https://api.twitter.com/1.1/geo/id/5a110d312052166f.json', place_type='city', name='San Francisco', full_name='San Francisco, CA', country_code='US', country='United States', centroid=[-122.4461400159226, 37.759828999999996], bounding_box=BoundingBox(_api=<tweepy.api.API object at 0x000001D647D236D8>, type='Polygon', coordinates=[[[-122.514926, 37.708075], [-122.514926, 37.833238], [-122.357031, 37.833238], [-122.357031, 37.708075], [-122.514926, 37.708075]]]), attributes={})], geometry=None, polylines=[], centroid=[-122.46598425785236, 37.79989625], bounding_box=BoundingBox(_api=<tweepy.api.API object at 0x000001D647D

In [17]:
tweet = api.home_timeline(count=1)[0]

In [ ]:
dir(tweet)

In [40]:
[tweet.id, tweet.created_at, tweet.text, tweet.user.followers_count, tweet.user.verified, tweet.possibly_sensitive, 
 tweet.coordinates, tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.entities['hashtags'], tweet.sc ]

[956781542491283456,
 datetime.datetime(2018, 1, 26, 6, 51, 10),
 'RT @RickyPo: Sub-Saharan nations lead the way in open access https://t.co/c7XgMmIZsp',
 684,
 False,
 False,
 None,
 None,
 1,
 0,
 []]

In [175]:
# api.geo_search
api.rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1516974077}}

After perusing a few tweets as well as news articles, the following keywords have been suggested

In [177]:
search_query = ('"day zero" water OR water crisis cape OR "day zero" crisis OR cape water OR cape drought')
max_tweets= 100000
tweets_per_query = 100

In [178]:
#with open('../data/tweets.json', mode='w') as f:  # do a check if file already exists as 'x' creates file each time
date = []
text = []
retweet_count = []
follower_count = []
location = []
tags = []
verified = []

tweets = tpy.Cursor(api.search, q=search_query).items(max_tweets)

for tweet in tweets:
    text.append(tweet.text)
    date.append(tweet.created_at)
    retweet_count.append(tweet.retweet_count)
    follower_count.append(tweet.user.followers_count)
    location.append(tweet.user.location)
    tags.append(tweet.entities['hashtags'])
    verified.append(tweet.user.verified)
    
tweet_data = pd.DataFrame({'text': text,
                          'date': date,
                          'retweet_count': retweet_count,
                          'follower_count': follower_count,
                          'location': location,
                          'tags': tags,
                          'verified': verified})
    
#ata = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

# data['date'] = np.array([tweet.created_at for tweet in tweets.items])
# data['location'] = np.array([tweet.place for tweet in tweets])
# data['retweet_count'] = np.array([tweet.retweet_count for tweet in tweets])
# data['follower_count'] = np.array([tweet.user.followers_count for tweet in tweets])
# data['tags'] = np.array([tweet.entities['hashtags'] for tweet in tweets])

,date,follower_count,location,retweet_count,tags,text,verified
0,2018-01-26 09:47:59,247,UK,0,"[{'text': 'water', 'indices': [62, 68]}]","Another commodity, alongside food, that we tak...",False
1,2018-01-26 09:47:59,737,Washington (the state),1,"[{'text': 'CapeTown', 'indices': [21, 30]}]",RT @TheDailyClimate: #CapeTown authorities are...,False
2,2018-01-26 09:45:10,15189,"Bozeman, Mont.",1,"[{'text': 'CapeTown', 'indices': [0, 9]}]",#CapeTown authorities are scrambling to persua...,False
3,2018-01-26 02:14:37,650,"Cape Town, South Africa",147,[],"RT @AssaadRazzouk: Day Zero: Within 4 Months, ...",False
4,2018-01-26 00:00:00,419,Japan,134,[],RT @Reuters: WATCH: Cape Town's authorities ar...,False


In [ ]:
# todo kparse location column nicely

def parse_location(string):
    
    """
    takes in a string and returns the best guess for a country name
    """
    